# Lesson 8.4: Building Agents with LangGraph (Agentic Workflows)

---

In previous lessons, we learned about **Agents** in LangChain, which enable Large Language Models (LLMs) to make decisions and use **Tools** to interact with the external world. We were also introduced to **LangGraph**, a powerful library for building stateful and conditional workflows. This lesson will combine these two concepts, guiding you on how to **build more complex Agents** by leveraging LangGraph's graph structure to implement **Agentic Workflows**, specifically the classic **ReAct (Reasoning and Acting)** loop.

## 1. Integrating LLM and Tools into LangGraph Nodes to Create Agents

To build an Agent in LangGraph, we will consider the LLM and Tools as **Nodes** in the graph.

* **LLM as a Reasoning Node:** The LLM is the "brain" of the Agent. In LangGraph, we will have one or more Nodes whose primary task is to call the LLM. This Node will receive the current state (including conversation history, intermediate steps, etc.), and the LLM will reason to decide:
    * What action needs to be taken (which Tool to call with what input)?
    * Or is there enough information to provide a final answer?
* **Tools as Action Nodes:** Each Tool (e.g., web search, calculator, custom API) will be encapsulated into a separate Node. This Node will receive the necessary input from the state (as decided by the LLM), execute the Tool, and return the observation result to update the state.
* **State:** The graph's state will store all necessary information for the Agent, including conversation history, the Agent's intermediate reasoning and action steps (agent scratchpad), and results from Tools.




---

## 2. Building a Basic Agent with LangGraph

A basic Agent in LangGraph typically follows an iterative pattern:

1.  **LLM Reasoning Node (`call_llm`):**
    * Receives the current state (including conversation history and previous intermediate steps).
    * Uses the LLM to generate a "Thought" and an "Action" or a "Final Answer."
    * Updates the state with the LLM's reasoning and action/answer.
    * **Important:** This Node will return a value to control a Conditional Edge, indicating the next step (e.g., "continue" to call a Tool, or "end" to finish).

2.  **Tool Calling Node (`call_tool`):**
    * Only activated if the LLM decides a Tool needs to be called.
    * Receives the state, extracts information about the Tool to call and its input.
    * Executes the chosen Tool.
    * Updates the state with the "Observation" (result of the Tool execution).

3.  **Observation Processing Node (Implicit in `call_llm` or `call_tool`):**
    * In the ReAct model, the observation result from the Tool will be fed back to the LLM in the next turn. This is usually handled by adding the Observation to `chat_history` or `agent_scratchpad` in the state, which the LLM Node then receives in its next call.


---

## 3. Implementing the Classic ReAct (Reasoning and Acting) Loop with LangGraph

The **ReAct loop** is one of the most common Agent architectures, where the LLM alternates between **Reasoning** and **Acting**.

* **Reasoning:** The LLM analyzes the request, conversation history, and previous observations to generate a "Thought" and decide the next "Action" (call a Tool or provide a final answer).
* **Acting:** The Agent executes the Tool chosen by the LLM and receives an "Observation" (the result).
* **Loop:** This Observation is then fed back to the LLM for it to continue reasoning and acting until the goal is achieved.

LangGraph is the perfect tool to implement the ReAct loop due to its state management and Conditional Edges capabilities:

1.  **State:** The `AgentState` will store `chat_history` (including `HumanMessage`, `AIMessage`, and `ToolMessage` - Observations).
2.  **Nodes:**
    * One Node for LLM reasoning and decision-making (call Tool or finish).
    * One Node for Tool execution.
3.  **Conditional Edge:** A conditional function will check the output of the LLM Node to decide whether the Agent should transition to the Tool execution Node or terminate. If it transitions to the Tool Node, after the Tool executes, the flow will return to the LLM Node to continue reasoning (forming a loop).




---

## 4. Practical Example: Building an Agent Capable of Web Search and Calculations

We will build a simple LangGraph Agent that can use a web search tool and a calculator tool to answer questions.

**Preparation:**
* Ensure you have the necessary libraries installed: `langchain-openai`, `google-search-results`, `numexpr`, `langgraph`.
* Set the `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai google-search-results numexpr langgraph

import os
from typing import TypedDict, Annotated, List, Union, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, END
import operator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain_community.tools.calculator.tool import Calculator
from langchain_core.agents import AgentFinish, AgentAction # To parse LLM output

# Set environment variables for OpenAI and SerpAPI keys
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY"

# --- 1. Define the Agent Graph's State Type ---
# chat_history: Conversation history, will be appended.
# intermediate_steps: Agent's intermediate steps (Thought, Action, Observation), also appended.
class AgentState(TypedDict):
    chat_history: Annotated[List[BaseMessage], operator.add]
    intermediate_steps: Annotated[List[Union[AgentAction, ToolMessage]], operator.add]

# --- 2. Initialize LLM and Tools ---
# Use LLM with low temperature for more consistent responses for the Agent.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Search tool
search_tool = Tool(
    name="Google Search",
    func=SerpAPIWrapper().run,
    description="Hữu ích khi bạn cần tìm kiếm thông tin trên Google về các sự kiện hiện tại hoặc dữ liệu thực tế." # Useful when you need to search for information on Google about current events or factual data.
)

# Calculator tool
calculator_tool = Calculator()

# Collection of Tools
tools = [search_tool, calculator_tool]

# --- 3. Define the Agent Prompt (to be used in the LLM Node) ---
# This prompt instructs the LLM on how to think and use tools.
# It includes chat history and agent_scratchpad for intermediate steps.
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý hữu ích. Bạn có quyền truy cập vào các công cụ sau: {tools}. Sử dụng chúng để trả lời các câu hỏi của người dùng. Nếu bạn đã có câu trả lời cuối cùng, hãy trả lời trực tiếp."), # You are a helpful assistant. You have access to the following tools: {tools}. Use them to answer user questions. If you have a final answer, respond directly.
    MessagesPlaceholder(variable_name="chat_history"), # Chat history
    MessagesPlaceholder(variable_name="agent_scratchpad"), # Thought, Action, Observation steps
])

# --- 4. Define the Graph Nodes ---

# Node 1: Call LLM (ReAct's reasoning/acting part)
def call_llm_node(state: AgentState) -> Dict[str, Any]:
    """
    This node calls the LLM to reason about the next step.
    The LLM can decide to call a Tool or provide a final answer.
    """
    print("\n--- Node: Call LLM (Reasoning/Acting) ---") # --- Node: Call LLM (Reasoning/Acting) ---
    messages = agent_prompt.format_messages(
        tools=tools, # Pass the list of tools to the prompt
        chat_history=state["chat_history"],
        agent_scratchpad=state["intermediate_steps"] # Previous intermediate steps
    )
    
    # Call LLM and parse output
    response = llm.invoke(messages)
    
    # LangChain has a built-in parser for Agent, but we'll do it manually
    # to illustrate how to check output for AgentAction or AgentFinish
    
    # If LLM decides to give a final answer
    if "Final Answer:" in response.content:
        final_answer_content = response.content.split("Final Answer:", 1)[1].strip()
        # Return AgentFinish to end the graph
        return {"chat_history": [AIMessage(content=final_answer_content)], "intermediate_steps": [AgentFinish(return_values={"output": final_answer_content}, log=response.content)]}
    
    # If LLM decides to call a Tool (parse AgentAction)
    try:
        # Try to parse the output into AgentAction
        # This is a complex part, LLM needs to return a specific format
        # Example: "Thought: ...\nAction: tool_name\nAction Input: tool_input"
        # For simplicity, we will assume LLM returns a parsable string
        # or you can use a specialized OutputParser.
        
        # A simple way to extract Action:
        thought_match = "Thought:"
        action_match = "Action:"
        action_input_match = "Action Input:"

        if thought_match in response.content and action_match in response.content and action_input_match in response.content:
            thought_part = response.content.split(thought_match, 1)[1].split(action_match, 1)[0].strip()
            action_part = response.content.split(action_match, 1)[1].split(action_input_match, 1)[0].strip()
            action_input_part = response.content.split(action_input_match, 1)[1].strip()

            action = AgentAction(tool=action_part, tool_input=action_input_part, log=response.content)
            print(f"  LLM decides to act: Tool='{action.tool}', Input='{action.tool_input}'") # LLM decides to act:
            return {"intermediate_steps": [action]}
        else:
            # If not Final Answer and cannot parse Action, consider it an error or unclear
            print(f"  LLM response unclear, neither Action nor Final Answer: {response.content}") # LLM response unclear, neither Action nor Final Answer:
            return {"chat_history": [AIMessage(content=f"Xin lỗi, tôi không hiểu yêu cầu của bạn. Phản hồi của tôi: {response.content}")], "intermediate_steps": [AgentFinish(return_values={"output": "LLM parsing error"}, log=response.content)]} # Sorry, I don't understand your request. My response:

    except Exception as e:
        print(f"  Error parsing LLM output to Action: {e}. LLM Response: {response.content}") # Error parsing LLM output to Action:
        # Return an error or end the flow
        return {"chat_history": [AIMessage(content=f"Xin lỗi, tôi gặp vấn đề khi xử lý yêu cầu của bạn: {e}")], "intermediate_steps": [AgentFinish(return_values={"output": "LLM parsing error"}, log=response.content)]} # Sorry, I encountered a problem processing your request:


# Node 2: Call Tool (ReAct's acting part)
def call_tool_node(state: AgentState) -> Dict[str, Any]:
    """
    This node executes the Tool that the LLM decided on.
    """
    print("--- Node: Call Tool (Execute tool) ---") # --- Node: Call Tool (Execute tool) ---
    last_action = state["intermediate_steps"][-1] # Get the last AgentAction
    
    tool_name = last_action.tool
    tool_input = last_action.tool_input

    # Find and run the Tool
    selected_tool = next((t for t in tools if t.name == tool_name), None)
    if selected_tool:
        try:
            tool_output = selected_tool.run(tool_input)
            print(f"  Tool '{tool_name}' returns: {tool_output[:100]}...") # Tool '{tool_name}' returns:
            # Return ToolMessage to add to intermediate_steps (Observation)
            return {"intermediate_steps": [ToolMessage(content=tool_output, tool_call_id=last_action.tool)]}
        except Exception as e:
            error_message = f"Lỗi khi thực thi Tool '{tool_name}' với đầu vào '{tool_input}': {e}" # Error executing Tool '{tool_name}' with input '{tool_input}':
            print(f"  {error_message}")
            return {"intermediate_steps": [ToolMessage(content=error_message, tool_call_id=last_action.tool)]}
    else:
        error_message = f"Không tìm thấy Tool: {tool_name}" # Tool not found:
        print(f"  {error_message}")
        return {"intermediate_steps": [ToolMessage(content=error_message, tool_call_id="unknown_tool")]}

# --- 5. Define the Conditional Edge function (decide to continue loop or end) ---
def should_continue(state: AgentState) -> str:
    """
    This function checks the last intermediate step to decide the next flow.
    If it's an AgentFinish, end. If it's an AgentAction, call the Tool.
    """
    last_step = state["intermediate_steps"][-1]
    if isinstance(last_step, AgentFinish):
        print("--- Decision: END (AgentFinish) ---") # --- Decision: END (AgentFinish) ---
        return "end" # End the graph
    elif isinstance(last_step, AgentAction):
        print("--- Decision: CONTINUE (AgentAction) ---") # --- Decision: CONTINUE (AgentAction) ---
        return "continue" # Continue the loop (call Tool)
    else:
        # Unexpected case, could be a ToolMessage without a preceding AgentAction
        # Or an invalid state.
        print(f"--- Decision: ERROR/UNKNOWN (Unexpected type: {type(last_step)}) ---") # --- Decision: ERROR/UNKNOWN (Unexpected type:
        return "end" # End to prevent infinite loop

# --- 6. Build the Agent Graph ---
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("call_llm", call_llm_node)
workflow.add_node("call_tool", call_tool_node)

# Set entry point: Always start by calling the LLM to reason
workflow.set_entry_point("call_llm")

# Define conditional edge from LLM Node
# LLM will decide whether to continue by calling a Tool or end
workflow.add_conditional_edges(
    "call_llm", # Node from which we branch
    should_continue, # Function that decides the path
    {
        "continue": "call_tool", # If 'continue', go to call_tool
        "end": END # If 'end', end the graph
    }
)

# Define edge from Tool Node back to LLM Node
# After the Tool is called, the flow returns to the LLM to continue reasoning with the Tool result
workflow.add_edge("call_tool", "call_llm")

# Compile the graph
app = workflow.compile()

print("\n--- Starting Agent with LangGraph Practical ---") # --- Starting Agent with LangGraph Practical ---

# --- Scenario 1: Question requires search and calculation ---
print("\n--- Scenario 1: Question requires search and calculation ---") # --- Scenario 1: Question requires search and calculation ---
initial_state_1 = {"chat_history": [HumanMessage(content="Thời tiết hôm nay ở London là bao nhiêu độ C? Sau đó nhân kết quả với 2.")]} # What is the weather like today in London in Celsius? Then multiply the result by 2.
final_state_1 = app.invoke(initial_state_1)
print(f"\nFinal response:") # Final response:
for message in final_state_1["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

# --- Scenario 2: Question can be answered directly ---
print("\n--- Scenario 2: Question can be answered directly ---") # --- Scenario 2: Question can be answered directly ---
initial_state_2 = {"chat_history": [HumanMessage(content="Thủ đô của Pháp là gì?")]} # What is the capital of France?
final_state_2 = app.invoke(initial_state_2)
print(f"\nFinal response:") # Final response:
for message in final_state_2["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

# --- Scenario 3: Simple calculation question ---
print("\n--- Scenario 3: Simple calculation question ---") # --- Scenario 3: Simple calculation question ---
initial_state_3 = {"chat_history": [HumanMessage(content="Tính 150 chia 3.")]} # Calculate 150 divided by 3.
final_state_3 = app.invoke(initial_state_3)
print(f"\nFinal response:") # Final response:
for message in final_state_3["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

print("\n--- End of Agent with LangGraph Practical ---") # --- End of Agent with LangGraph Practical ---
